In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
property_data = pd.read_csv('../../properati-AR-2017-08-01-properties-sell.csv')

# ANALISIS DE LAS DESCRIPCIONES

In [3]:
#Primero vamos a ver cuantas propiedades tienen descripciones y cuantas titulo.
property_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 27 columns):
id                            187482 non-null object
created_on                    187482 non-null object
operation                     187482 non-null object
property_type                 187482 non-null object
place_name                    187447 non-null object
place_with_parent_names       187482 non-null object
country_name                  187482 non-null object
state_name                    187482 non-null object
geonames_id                   158318 non-null float64
lat-lon                       118761 non-null object
lat                           118761 non-null float64
lon                           118761 non-null float64
price                         154443 non-null float64
currency                      154441 non-null object
price_aprox_local_currency    154443 non-null float64
price_aprox_usd               154443 non-null float64
surface_total_in_m2           12502

## Analisis de los titulos.

In [4]:
def parseoTipo(x):
    
    tipo = ""
    tiposViviendas = ['casa', 'ph','p.h.', 'departamento','depto','dpto','dto',
                      'chalet','duplex','duples','triplex','edificio','semipiso','piso',
                      'p.baja','planta baja', 'p.b.','casona', 'loft', 'cabaña', 'mansion',
                      'local','quinta','bungalow','oficina',
                      'terreno','lote']
    x=x.lower()
    
    for i in tiposViviendas:
        aux = ""
        
        if( x.find(i) >= 0 ):
            aux += i
        if( aux != "" and aux not in tipo):
            tipo+=aux
            tipo+=" "

    if(tipo !=""): return tipo
    return x

def identificarCaracteristica(x,j,indice,aux,caract):
            
    if( j not in caract ):

        "Si son ambientes, vemos cuantos"
        if(j == 'ambiente' or j== 'amb'):
            aux = x[indice - 1]
            aux += " "

        "Si son dormitorios, vemos cuantos"
        if(j=="dormitorios" or j == "dormitorio" or j == "dorm" or j == "hab" or j == "habitacion"):
            aux = x[indice - 1]
            aux += " "

        aux += j
        caract.append( aux )

def actualizarPosicion(j):
    if(j<=3):#condiciones
        j=4
    if(j>3 and j<=6):#ambientes
        j=7
    if(j>6 and j<=13):#cuartos
        j=14
    if(j>13 and j<=17):#cochera
        j=18

def chequearFin(caract,j):
    if( j == 3 or j== 6 or j==13 or j>=17 ):
        caract.append(None)
        
def parseoCaracteristica(x):
    
    caract = []
    caractViviendas = ['estrenar','nueva', 'condiciones','refaccionar',
                       'monoambiente','ambiente','amb',
                       'dormitorio','dorm','dormitorios','hab','habitacion','cuarto','cuartos',
                       'estac', 'estacionamiento','cochera','garage',
                       
                       'living','comedor','cocina','baño','lavadero','piso',           
                       'balcon','pileta','jardin','patio','parrilla']
    
    if( x == None or x == ""):
        return 
    
    if type( x ) is str:
        x = x.lower()
    if type( x ) is float:
        x = str(x).lower()
    
    x = x.split()
    
    for j in xrange(len(caractViviendas)):
        aux = ""
        palabraAct = caractViviendas[j]
        
        if( palabraAct in x ):
            indice = x.index(palabraAct)
            identificarCaracteristica(x,palabraAct,indice,aux,caract)
            actualizarPosicion(j)
        
        else:
            for ind in xrange(len(x)):
                if(palabraAct in x[ind]):
                    identificarCaracteristica(x,palabraAct,ind,aux,caract)
                    actualizarPosicion(j)
                else:
                    chequearFin(caract,j)
                    
                    
    if( len(caract) != 0 ): return caract
    #print x
    return None

In [6]:
#Primero limpiamos las columnas que no es posible calcularle el precio.
print len(property_data)

propertyData = property_data[  (property_data['price_aprox_usd'].isnull() == False)  ]
           #or (property_data['price_per_m2'].isnull == False 
            #and property_data['surface_total_in_m2'] == False )]
#faltaria recortar con eso pero ya es una buena aprox.

print len(propertyData)


#Parseo el titulo y me quedo con las caracteristicas.
caracteristicas = propertyData[['id','title']]
caracteristicas['title'] = caracteristicas['title'].apply(lambda x: parseoCaracteristica(x))
#print caracteristicas.head(5)


#Parseo el titulo y me quedo con el tipo de vivienda.
#tiposVivienda = titulos[['title', 'id', 'price_per_m2']]
#tiposVivienda['title'] = tiposVivienda['title'].apply(lambda x: parseoTipo(x))

#Mergeo los dos separados
#completo = pd.merge(caracteristicas, tiposVivienda, on='id', how='inner')
#completo.rename(columns={'title_x':'caracteristicas'}, inplace=True)
#completo.rename(columns={'title_y':'tipo'}, inplace=True)

#completo.head(10)

"""
HAY MUY POCOS QUE TIENEN CARACTERISTICAS. HAY QUE VER SI CON LA DESCRIPCION PODEMOS EXPANDIR
UN POCO MAS LA INFO. ES MUY POCA.
"""


187482
154443


/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'\nHAY MUY POCOS QUE TIENEN CARACTERISTICAS. HAY QUE VER SI CON LA DESCRIPCION PODEMOS EXPANDIR\nUN POCO MAS LA INFO. ES MUY POCA.\n'

## Ahora parseo la descripcion

In [7]:
descripcion  = propertyData[['id','description']]

print len(descripcion)#154443

#Primera pasada super basica, vamos a refinar para conseguir mas datos
lDescripcionesN = descripcion['description'].apply(lambda x: parseoCaracteristica(x))
lDescripcionesN = lDescripcionesN[lDescripcionesN.isnull() == False]
#Para esta linea poner return None, solo lo hice para ver cuantos no tienen descripcion
print len(lDescripcionesN)#123212

154443
154443


In [8]:
#Vamos a intentar refinar un poco
lDescrip = descripcion['description'].apply(lambda x: parseoCaracteristica(x))
print lDescrip.head(10)
#print len(lDescrip)#123212

0     [None, None, None, None, None, None, None, Non...
1     [None, None, None, None, None, None, None, Non...
4     [None, None, None, None, None, None, None, Non...
5     [None, None, None, None, None, None, None, Non...
6     [None, None, None, None, None, None, None, Non...
8     [None, None, None, None, None, None, None, Non...
9     [None, None, None, None, None, None, None, Non...
10    [None, None, None, None, None, None, None, Non...
11    [None, None, None, None, None, None, None, Non...
12    [None, None, None, None, None, None, None, Non...
Name: description, dtype: object


# Vamos a separar las caracteristicas 

In [12]:
condicion = lDescrip.apply(lambda row: row[0])
print condicion.isnull().sum()

ambientes = lDescrip.apply(lambda row: row[1])
print ambientes.isnull().sum()

habitaciones = lDescrip.apply(lambda row: row[2])
print habitaciones.isnull().sum()

garage = lDescrip.apply(lambda row: row[3])
print garage.isnull().sum()

132743
153533
154345
154391
